In [1]:
%load_ext autoreload
%autoreload 2

from copy import deepcopy
from typing import List
import numpy as np
import sklearn
from celer import Lasso
import random
from sklearn import datasets
from sklearn.base import BaseEstimator, RegressorMixin, ClassifierMixin
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier,export_text
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
from scipy.linalg import hadamard
from synthetic_combinations import *
from scipy.sparse import random
from scipy import stats
from numpy.linalg import matrix_rank
from sklearn.preprocessing import normalize
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression

# Generate Test Data  

## Define Parameters

In [5]:
N = 200
p = 15
r  = 3
s = p**2


## Generate Alpha Matrix and Potential Outcome Matrix

In [6]:
# Cell for generating alpha matrix
rvs = stats.norm().rvs
rank_users = []
for i in range(r):
    alpha_u = random(2**p,1,density=(s)/(2**p),data_rvs = rvs).toarray()
    rank_users.append(alpha_u[:,0])
rank_users = np.array(rank_users)
B = np.random.dirichlet([1]*r, 200 - r)
remaining_users = np.matmul(B,rank_users)
all_users = np.concatenate((rank_users,remaining_users))
all_users = normalize(all_users, axis=1, norm='l2')

In [7]:
# Cell for generating potential outcome matrix 
fourier_characteristic_matrix = hadamard(2**p)
potential_outcome_matrix = np.matmul(all_users,fourier_characteristic_matrix)
#min_max_scaler = preprocessing.MinMaxScaler(feature_range = (-1,1))
#potential_outcome_matrix = min_max_scaler.fit_transform(potential_outcome_matrix)

## Generate Observation Pattern and Observation Matrix from Uniform Sampling

In [8]:
#Generate indices for donor units 
from random import choices
num_donor_units = 2*r
num_donor_unit_observations = 3000
num_non_donor_unit_observations = r**4
donor_unit_observations = choices(range(2**p), k = num_donor_unit_observations)
non_donor_unit_observations = choices(range(2**p), k = num_non_donor_unit_observations)

In [9]:
observation_matrix = np.empty((N,2**p,))
observation_matrix[:] = np.nan

In [10]:
observation_matrix[:num_donor_units,donor_unit_observations] = potential_outcome_matrix[:num_donor_units,donor_unit_observations]
observation_matrix[num_donor_units:,non_donor_unit_observations] =  potential_outcome_matrix[num_donor_units:,non_donor_unit_observations]
#donor_unit_observation_matrix = 
observation_matrix.shape        

(200, 32768)

# Synthetic Combinations 

In [11]:
sc = synth_combo()

## Do Horizontal Regression 

In [12]:
donor_unit_indices = [i for i in range(N)]
horizontal_regression_observation_matrix = sc.horizontal_fit(observation_matrix,donor_unit_indices)

100%|██████████████████████████████████████████████████████████████████████████| 200/200 [21:55<00:00,  6.58s/it]


In [15]:
np.linalg.norm(horizontal_regression_observation_matrix[:6,:] - potential_outcome_matrix[:6,:])**2/(N*potential_outcome_matrix.shape[1])

0.0009759161636400511

In [27]:
donor_unit_imputed_matrix = horizontal_regression_observation_matrix[donor_unit_indices,:]
np.linalg.norm(donor_unit_imputed_matrix - potential_outcome_matrix[donor_unit_indices,:])**2/(len(donor_unit_indices)*donor_unit_imputed_matrix.shape[1])

0.04089749099345411

## Vertical Regression 

In [28]:
vertical_regression_observation_matrix = sc.vertical_fit(horizontal_regression_observation_matrix,donor_unit_indices,num_components = 3)

100%|█████████████████████████████████████████████████████████████████████████| 194/194 [00:00<00:00, 350.31it/s]


In [29]:
np.linalg.norm(vertical_regression_observation_matrix - potential_outcome_matrix)**2/(N*donor_unit_imputed_matrix.shape[1])

0.0094417009015897

# Only Horizontal Regression 

In [ ]:
unit_indices = [i for i in range(N)][num_donor_units:]
lasso_regression_observation_matrix = sc.horizontal_fit(horizontal_regression_observation_matrix,unit_indices)

  0%|                                                                                    | 0/194 [00:00<?, ?it/s]